# Driving Dust Emissions with GFS Data

This notebook demonstrates how to use `pyfengsha` to calculate dust emissions using meteorological data from the NOAA Global Forecast System (GFS).

We will:
1.  Download a sample GFS GRIB2 file from NOAA.
2.  Read the data using `xarray` and `cfgrib`.
3.  Prepare the necessary inputs for the FENGSHA model (mocking static soil fields for this example).
4.  Run the dust emission model.
5.  Visualize the results.


In [ ]:
import os
import requests
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime, timedelta

# Import the FENGSHA xarray interface
from pyfengsha.xarray_interface import DustEmissionFENGSHA_xr

## 1. Download GFS Data

We will download a single time step of GFS data from the NOAA S3 bucket. We look for the 0.25 degree resolution file. We use yesterday's date to ensure data availability.


In [ ]:
# Configuration
# Use yesterday's date to ensure file exists on S3
date_target = datetime.utcnow() - timedelta(days=1)
GFS_DATE = date_target.strftime("%Y%m%d") # YYYYMMDD
GFS_CYCLE = "00"      # Cycle (00, 06, 12, 18)
GFS_FILENAME = f"gfs.{GFS_DATE}.t{GFS_CYCLE}z.pgrb2.0p25.f000"

# URL for NOAA GFS S3 bucket
GFS_URL = f"https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.{GFS_DATE}/{GFS_CYCLE}/atmos/gfs.t{GFS_CYCLE}z.pgrb2.0p25.f000"

def download_gfs_data(url, filename):
    if os.path.exists(filename):
        print(f"{filename} already exists.")
        return

    print(f"Downloading {filename} from {url}...")
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("Download complete.")
    except Exception as e:
        print(f"Error downloading file: {e}")

download_gfs_data(GFS_URL, GFS_FILENAME)

## 2. Read Data with Xarray and Cfgrib

We use `xarray` with the `cfgrib` engine to read the GRIB2 file. Note that GRIB files often contain multiple vertical coordinate systems (surface, isobaric, etc.), so we may need to filter by keys to open the dataset cleanly.

We need:
*   **Friction Velocity (ustar)**: Often parameter `ust` or similar at surface.
*   **Soil Moisture**: Volumetric soil moisture (e.g., `swvl1` for 0-10cm layer).


In [ ]:
try:
    # Open surface level data
    # We filter by typeOfLevel='surface' or 'depthBelowLandLayer' depending on variable
    # Friction velocity is usually 'surface'
    ds_surf = xr.open_dataset(
        GFS_FILENAME, 
        engine='cfgrib', 
        backend_kwargs={'filter_by_keys': {'typeOfLevel': 'surface'}}
    )
    
    # Soil moisture is often in 'depthBelowLandLayer'
    ds_soil = xr.open_dataset(
        GFS_FILENAME, 
        engine='cfgrib', 
        backend_kwargs={'filter_by_keys': {'typeOfLevel': 'depthBelowLandLayer', 'stepType': 'instant'}}
    )

    print("Datasets opened successfully.")
    
    # Extract variables (names depend on cfgrib/eccodes mapping)
    # Check ds.data_vars to see actual names if unsure
    
    # Friction Velocity
    # GFS often names it 'ust'
    ustar_da = ds_surf['ust'] 

    # Volumetric Soil Moisture (0-0.1m)
    # GFS often names it 'swvl1' or similar
    slc_da = ds_soil['swvl1']
    
    # Ensure they are on the same grid (they should be for the same GFS file)
    # We might need to select the specific soil layer if multiple are present in the file
    
except Exception as e:
    print(f"Could not open GRIB file (likely due to missing cfgrib/eccodes or file): {e}")
    print("Generating MOCK GFS data for demonstration...")
    
    # Mock Grid (Global 1x1)
    lat = np.linspace(-90, 90, 181)
    lon = np.linspace(0, 360, 361)
    dims = ('lat', 'lon')
    coords = {'lat': lat, 'lon': lon}
    
    # Mock USTAR (high in some bands)
    ustar_val = 0.2 + 0.5 * np.sin(np.linspace(-3, 3, 181)[:, None])**2 * np.ones((181, 361))
    ustar_da = xr.DataArray(ustar_val, coords=coords, dims=dims, name='ust')
    
    # Mock Soil Moisture (dry bands)
    soilw_val = 0.3 * np.abs(np.cos(np.linspace(-3, 3, 181)[:, None])) * np.ones((181, 361))
    soilw_val[60:80, :] = 0.05 # Dry band around 30N
    slc_da = xr.DataArray(soilw_val, coords=coords, dims=dims, name='swvl1')

# Visualize inputs
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ustar_da.plot(ax=ax[0], cmap='inferno')
ax[0].set_title('Friction Velocity (m/s)')
slc_da.plot(ax=ax[1], cmap='Blues')
ax[1].set_title('Soil Moisture (m3/m3)')
plt.show()

## 3. Prepare Model Inputs

The FENGSHA model requires several static fields (soil texture, land mask, etc.) in addition to meteorology. For this example, we will mock these static fields or set them to constant values. In a real application, you would regrid high-resolution datasets (like FAO soil maps) to the GFS grid.


In [ ]:
# Align all inputs to the meteorological grid
ustar = ustar_da
slc = slc_da

# 1. Land Mask (ORO) - 1 for Land
# Simple approximation: assume land where soil moisture is defined or manually set
oro = xr.ones_like(ustar) 

# 2. Soil Texture (Clay, Sand, Silt fractions)
# We set a "Sandy Loam" type everywhere for this example
# Real data sources: STATSGO, FAO
clay = xr.full_like(ustar, 0.2)  # 20% Clay
sand = xr.full_like(ustar, 0.5)  # 50% Sand
silt = xr.full_like(ustar, 0.3)  # 30% Silt

# 3. Sediment Supply Map (SSM)
# Factor 0-1 indicating availability of erodible sediment
ssm = xr.ones_like(ustar)

# 4. Drag Partition (Roughness)
# Lower values (< ~0.1) allow emission. High values (vegetation, rocks) suppress it.
rdrag = xr.full_like(ustar, 0.01)

# 5. Air Density
airdens = xr.full_like(ustar, 1.225)

# 6. Vegetation & LAI
# Used in advanced drag partitioning options. We'll zero them out for now.
vegfrac = xr.zeros_like(ustar)
lai = xr.zeros_like(ustar)

# 7. Threshold Velocity Map
# Baseline threshold friction velocity (often ~0.2 - 0.4 m/s)
uthrs = xr.full_like(ustar, 0.3) 

# 8. Particle Size Distribution
# The model outputs emissions for specified particle size bins.
# Let's define 4 bins.
nbins = 4
# Particle density (kg/m3)
rhop = xr.DataArray(np.full(nbins, 2650.0), dims='bin', name='rhop')
# Mass fraction of each bin (must sum to 1)
distribution = xr.DataArray(np.array([0.1, 0.3, 0.4, 0.2]), dims='bin', name='distribution')

# 9. Other Scalar Constants
alpha = 1.0       # Global tuning constant
gamma = 1.0       # Emission exponential factor
kvhmax = 1.0      # Max vertical-to-horizontal flux ratio
grav = 9.81
drylimit_factor = 1.0
moist_correct = 1.0
drag_opt = 0      # 0 = Use rdrag directly

## 4. Run the Dust Emission Model

We call the `DustEmissionFENGSHA_xr` wrapper. This function applies the Numba-accelerated kernel over the xarray grid efficiently.


In [ ]:
print("Running FENGSHA Dust Emission Model...")
emissions = DustEmissionFENGSHA_xr(
    fraclake=xr.zeros_like(ustar),
    fracsnow=xr.zeros_like(ustar),
    oro=oro,
    slc=slc,
    clay=clay,
    sand=sand,
    silt=silt,
    ssm=ssm,
    rdrag=rdrag,
    airdens=airdens,
    ustar=ustar,
    vegfrac=vegfrac,
    lai=lai,
    uthrs=uthrs,
    alpha=alpha,
    gamma=gamma,
    kvhmax=kvhmax,
    grav=grav,
    rhop=rhop,
    distribution=distribution,
    drylimit_factor=drylimit_factor,
    moist_correct=moist_correct,
    drag_opt=drag_opt
)
print("Calculation complete.")
print(f"Output shape: {emissions.shape} (lat, lon, bin)")


## 5. Visualize Results

We sum the emissions across all bins to get the total dust flux and plot it on a map.


In [ ]:
# Sum over all particle size bins
total_emission = emissions.sum(dim='bin')

# Plot
plt.figure(figsize=(12, 7))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Plot data, masking zeros for better visualization
total_emission.where(total_emission > 0).plot(
    ax=ax, 
    transform=ccrs.PlateCarree(),
    cmap='YlOrRd',
    cbar_kwargs={'label': 'Dust Flux (kg/m$^2$/s)'},
    robust=True # Handles outliers in color scaling
)

plt.title('Total Dust Emission Flux')
plt.show()